In [4]:
#Install Packages
#!pip install faiss-cpu
#!pip install sentence-transformers

In [1]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


## Step 1 : Create source embeddings (vectors) for the text column

In [9]:
from sentence_transformers import SentenceTransformer #From HuggingFace

In [6]:
df.text

0                                          Meditation and yoga can improve mental health
1                       Fruits, whole grains and vegetables helps control blood pressure
2                                      These are the latest fashion trends for this week
3                                      Vibrant color jeans for male are becoming a trend
4                                                     The concert starts at 7 PM tonight
5                        Navaratri dandiya program at Expo center in Mumbai this october
6                                      Exciting vacation destinations for your next trip
7    Maldives and Srilanka are gaining popularity in terms of low budget vacation places
Name: text, dtype: object

In [5]:
encoder = SentenceTransformer("all-mpnet-base-v2") # name of the model/transformer
vectors = encoder.encode(df.text)

In [7]:
vectors.shape

(8, 768)

In [8]:
dimension = vectors.shape[1]
dimension

768

## Step 2 : Build a FAISS Index for vectors

In [10]:
import faiss

index = faiss.IndexFlatL2(dimension) #index that uses Euclidean distance. 
# Creates a vector db that allows to do faster search

In [11]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x1750de850> >

## Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [12]:
index.add(vectors)

## Step 4 : Encode search text using same encoder and normalize the output vector

In [13]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [14]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape # now there's one more outer array (2 dimensional vector) index.search expects a 2-D array 

(1, 768)

## Step 5: Search for similar vector in the FAISS index created

In [16]:
distances, index_in_df = index.search(svec, k=2)
print(distances)
print("\n")
print(index_in_df)

[[1.3844838 1.4039094]]


[[3 2]]


In [17]:
df.loc[index_in_df[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [18]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query